# Conversational agent 💬

## Introduction 
This notebook demonstrates how to `create a conversational agent using LangChain`, OpenAI, and various tools. The agent can perform tasks such as fetching the current temperature for given coordinates and searching Wikipedia for page summaries.

### Setup the Environment, OpenAI API Key  and Imports
First, we need to import the necessary libraries and set up the Environemnt and the OpenAI API key:

In [1]:
# Import necessary libraries
import os
import openai
from dotenv import load_dotenv, find_dotenv

# Load environment variables from .env file
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

# Print OpenAI API key (masked)
print(f"OPENAI_API_KEY: {os.getenv('OPENAI_API_KEY')[:5]}*****")

OPENAI_API_KEY: sk-ft*****


**Note** Ensure you have the required packages installed:
```py
%pip install pydantic==1.10.8
%pip install rich
```

In [2]:
# Import necessary modules from Pydantic 
from typing import List
from pydantic import BaseModel, Field

Next, we import the tool decorator from LangChain:

In [3]:
# Import necessary modules from  Langchain
from langchain.tools import tool

### Creating Tools

#### Weather Tool
Now, let's define a tool to fetch the current temperature using the OpenMeteo API. We will create an input schema using Pydantic and define the function to make the API request:

In [4]:
import requests
#from pydantic import BaseModel, Field
import datetime

# Define the input schema for the OpenMeteo tool
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

   # Make the request to the OpenMeteo API
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    # Find the temperature closest to the current UTC time
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

#### Wikipedia Search Tool
We also need a tool to search Wikipedia and get page summaries:

In [5]:
import wikipedia

@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

We compile the list of available tools:

In [6]:
# List of available tools
tools = [get_current_temperature, search_wikipedia]

### Setting Up the Conversational Model
Next, we set up the ChatOpenAI model and create a chain that combines the prompt, model, and output parser:

In [7]:
# Import necessary modules from  Langchain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [8]:
# Format tools for OpenAI functions
functions = [format_tool_to_openai_function(f) for f in tools]

# Initialize the ChatOpenAI model with the formatted functions
model = ChatOpenAI(temperature=0).bind(functions=functions)

# Create a prompt template for the chat model
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])

# Create a chain combining the prompt, model, and output parser
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

C:\Users\Michela\AppData\Local\Temp\ipykernel_5296\2133018398.py:2: LangChainDeprecationWarning: The function `_format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use :meth:`~langchain_core.utils.function_calling.convert_to_openai_function()` instead.
  functions = [format_tool_to_openai_function(f) for f in tools]
C:\Users\Michela\AppData\Local\Temp\ipykernel_5296\2133018398.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI(temperature=0).bind(functions=functions)


We can now invoke the chain with a user input to get the result:

In [9]:
# Invoke the chain with a user input
result = chain.invoke({"input": "what is the weather is Amsterdam?"})

We can check the tool used and the input parameters:

In [10]:
# Output the name of the tool used
result.tool

'get_current_temperature'

In [11]:
# Output the input parameters used by the tool
result.tool_input

{'latitude': 52.3676, 'longitude': 4.9041}

### Enhancing the Agent with an Agent Scratchpad
To improve the agent's functionality, we add an agent scratchpad to the prompt template:

In [12]:
from langchain.prompts import MessagesPlaceholder

# Update the prompt template to include an agent scratchpad
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

We create a new chain with the updated prompt and invoke it with an empty agent scratchpad:

In [13]:
# Create a new chain with the updated prompt
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [14]:
# Invoke the updated chain with a user input and an empty agent scratchpad
result1 = chain.invoke({
    "input": "what is the weather is Amsterdam?",
    "agent_scratchpad": []
})

In [15]:
# Output the name of the tool used in the updated chain
result1.tool

'get_current_temperature'

Next, we run the get_current_temperature tool with the input parameters and store the observation

In [16]:
# Run the get_current_temperature tool with the input parameters and store the observation
observation = get_current_temperature(result1.tool_input)

C:\Users\Michela\AppData\Local\Temp\ipykernel_5296\2812115652.py:2: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  observation = get_current_temperature(result1.tool_input)
C:\Users\Michela\AppData\Local\Temp\ipykernel_5296\2904686110.py:32: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc_time = datetime.datetime.utcnow()


In [17]:
# Output the observation result
observation

'The current temperature is 6.3°C'

We can check the type of the result from the updated chain:

In [18]:
# Check the type of the result from the updated chain
type(result1)

langchain_core.agents.AgentActionMessageLog

To format the intermediate steps for OpenAI functions, we use the format_to_openai_functions function:

In [19]:
# Import necessary function to format intermediate steps for OpenAI functions
from langchain.agents.format_scratchpad import format_to_openai_functions

In [20]:
# Output the message log of the result
result1.message_log

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":52.3676,"longitude":4.9041}', 'name': 'get_current_temperature'}}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 112, 'total_tokens': 138, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-19af176a-a423-4efb-86b9-2c794bb58f7e-0')]

In [21]:
# Format the intermediate steps for OpenAI functions
format_to_openai_functions([(result1, observation), ])

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":52.3676,"longitude":4.9041}', 'name': 'get_current_temperature'}}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 112, 'total_tokens': 138, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-19af176a-a423-4efb-86b9-2c794bb58f7e-0'),
 FunctionMessage(content='The current temperature is 6.3°C', additional_kwargs={}, response_metadata={}, name='get_current_temperature')]

We invoke the chain again with the formatted intermediate steps:

In [22]:
# Invoke the chain again with the formatted intermediate steps
result2 = chain.invoke({
    "input": "what is the weather is Amsterdam?", 
    "agent_scratchpad": format_to_openai_functions([(result1, observation)])
})

In [23]:
# Output the final result
result2

AgentFinish(return_values={'output': 'The current temperature in Amsterdam is 6.3°C.'}, log='The current temperature in Amsterdam is 6.3°C.')

### Running the Agent in a Loop
We define a function to run the agent in a loop until a final result is obtained:

In [24]:
from langchain.schema.agent import AgentFinish

# Define a function to run the agent in a loop until a final result is obtained
def run_agent(user_input):
    intermediate_steps = []
    while True:
        result = chain.invoke({
            "input": user_input, 
            "agent_scratchpad": format_to_openai_functions(intermediate_steps)
        })
        if isinstance(result, AgentFinish):
            return result
        tool = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }[result.tool]
        observation = tool.run(result.tool_input)
        intermediate_steps.append((result, observation))

We create a runnable chain that includes the agent scratchpad:

In [25]:
from langchain.schema.runnable import RunnablePassthrough

# Create a runnable chain that includes the agent scratchpad
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | chain

We update the run_agent function to use the runnable chain:

In [26]:
# Define the run_agent function to use the runnable chain
def run_agent(user_input):
    intermediate_steps = []
    while True:
        result = agent_chain.invoke({
            "input": user_input, 
            "intermediate_steps": intermediate_steps
        })
        if isinstance(result, AgentFinish):
            return result
        tool = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }[result.tool]
        observation = tool.run(result.tool_input)
        intermediate_steps.append((result, observation))

We test the run_agent function with various inputs:

In [27]:
# Test the run_agent function with a sample input
run_agent("what is the weather is Amsterdam?")

C:\Users\Michela\AppData\Local\Temp\ipykernel_5296\2904686110.py:32: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc_time = datetime.datetime.utcnow()


AgentFinish(return_values={'output': 'The current temperature in Amsterdam is 6.3°C.'}, log='The current temperature in Amsterdam is 6.3°C.')

In [28]:
# Test the run_agent function with another sample input
run_agent("what is langchain?")

AgentFinish(return_values={'output': 'I couldn\'t find specific information about "Langchain." It seems like it might not be a widely recognized term or topic. If you have any other questions or need assistance with something else, feel free to ask!'}, log='I couldn\'t find specific information about "Langchain." It seems like it might not be a widely recognized term or topic. If you have any other questions or need assistance with something else, feel free to ask!')

In [29]:
# Test the run_agent function with a greeting input
run_agent("hi!")

AgentFinish(return_values={'output': 'Well, hello there! How can I assist you today?'}, log='Well, hello there! How can I assist you today?')

### Creating an Agent Executor
We create an AgentExecutor to manage the agent chain and tools:

In [30]:
from langchain.agents import AgentExecutor

# Create an AgentExecutor to manage the agent chain and tools
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True)

We test the agent_executor with various inputs:

In [31]:
# Test the agent_executor with a sample input
agent_executor.invoke({"input": "what is langchain?"})



> Entering new AgentExecutor chain...

Invoking: `search_wikipedia` with `{'query': 'Langchain'}`


Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.



Page: Milvus (vector database)
Summary: }}
{{Multiple issues|

Milvus is a distributed vector database developed by Zilliz. It is available as both open-source software and a cloud service.
Milvus is an open-source project under LF AI & Data Foundation distributed under the Apache License 2.0.



Page: Retrieval-augmented generation
Summary: Retrieval-Augmented Generation (RAG) is a technique that grants generative artificial intelligence models information retrieval capabilities. It modifies interactions with a large language model (LLM) so 

{'input': 'what is langchain?',
 'output': 'I couldn\'t find specific information about "Langchain." It seems like it might not be a widely recognized term or topic. If you have any other questions or need assistance with something else, feel free to ask!'}

In [32]:
# Test the agent_executor with another sample input
agent_executor.invoke({"input": "my name is Jane"})



> Entering new AgentExecutor chain...
Nice to meet you, Jane! How can I assist you today?

> Finished chain.


{'input': 'my name is Jane',
 'output': 'Nice to meet you, Jane! How can I assist you today?'}

In [33]:
# Test the agent_executor with a personal question
agent_executor.invoke({"input": "what is my name"})



> Entering new AgentExecutor chain...
I'm sorry, I don't have access to your personal information. How can I assist you today?

> Finished chain.


{'input': 'what is my name',
 'output': "I'm sorry, I don't have access to your personal information. How can I assist you today?"}

### Adding Conversation Memory
We update the prompt template to include chat history and create a new agent chain with memory support:

In [34]:
# Update the prompt template to include chat history
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [35]:
# Create a new agent chain with the updated prompt and model
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | prompt | model | OpenAIFunctionsAgentOutputParser()

In [36]:
from langchain.memory import ConversationBufferMemory

# Create a conversation buffer memory to store chat history
memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")

C:\Users\Michela\AppData\Local\Temp\ipykernel_5296\1094522753.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")


In [37]:
# Create a new AgentExecutor with memory support
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True, memory=memory)

We test the new agent_executor with memory support:

In [38]:
# Test the new agent_executor with memory support
agent_executor.invoke({"input": "my name is Jane"})



> Entering new AgentExecutor chain...
Nice to meet you, Jane! How can I assist you today?

> Finished chain.


{'input': 'my name is Jane',
 'chat_history': [HumanMessage(content='my name is Jane', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, Jane! How can I assist you today?', additional_kwargs={}, response_metadata={})],
 'output': 'Nice to meet you, Jane! How can I assist you today?'}

In [39]:
# Test the new agent_executor with a follow-up question
agent_executor.invoke({"input": "whats my name"})



> Entering new AgentExecutor chain...
You just told me your name is Jane. So, your name is Jane! How can I assist you further, Jane?

> Finished chain.


{'input': 'whats my name',
 'chat_history': [HumanMessage(content='my name is Jane', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, Jane! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='whats my name', additional_kwargs={}, response_metadata={}),
  AIMessage(content='You just told me your name is Jane. So, your name is Jane! How can I assist you further, Jane?', additional_kwargs={}, response_metadata={})],
 'output': 'You just told me your name is Jane. So, your name is Jane! How can I assist you further, Jane?'}

In [40]:
# Test the new agent_executor with a weather query
agent_executor.invoke({"input": "whats the weather in Amsterdam?"})



> Entering new AgentExecutor chain...

Invoking: `get_current_temperature` with `{'latitude': 52.3676, 'longitude': 4.9041}`




C:\Users\Michela\AppData\Local\Temp\ipykernel_5296\2904686110.py:32: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc_time = datetime.datetime.utcnow()


The current temperature is 6.3°CThe current temperature in Amsterdam is 6.3°C. Is there anything else you would like to know?

> Finished chain.


{'input': 'whats the weather in Amsterdam?',
 'chat_history': [HumanMessage(content='my name is Jane', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, Jane! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='whats my name', additional_kwargs={}, response_metadata={}),
  AIMessage(content='You just told me your name is Jane. So, your name is Jane! How can I assist you further, Jane?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='whats the weather in Amsterdam?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The current temperature in Amsterdam is 6.3°C. Is there anything else you would like to know?', additional_kwargs={}, response_metadata={})],
 'output': 'The current temperature in Amsterdam is 6.3°C. Is there anything else you would like to know?'}

### Creating a Custom Tool
Finally, we add a custom tool and update the list of available tools:

#### Create a chatbot

In [41]:
@tool
def create_your_own(query: str) -> str:
    """This function can do whatever you would like once you fill it in """
    print(type(query))
    return query[::-1]

In [42]:
# Add the new tool to the list of available tools
tools = [get_current_temperature, search_wikipedia, create_your_own]

In [43]:
import panel as pn  # GUI
pn.extension()
import panel as pn
import param

class cbfs(param.Parameterized):
    
    def __init__(self, tools, **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.functions = [format_tool_to_openai_function(f) for f in tools]
        self.model = ChatOpenAI(temperature=0).bind(functions=self.functions)
        self.memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "You are helpful but sassy assistant"),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])
        self.chain = RunnablePassthrough.assign(
            agent_scratchpad = lambda x: format_to_openai_functions(x["intermediate_steps"])
        ) | self.prompt | self.model | OpenAIFunctionsAgentOutputParser()
        self.qa = AgentExecutor(agent=self.chain, tools=tools, verbose=False, memory=self.memory)
    
    def convchain(self, query):
        if not query:
            return
        inp.value = ''
        result = self.qa.invoke({"input": query})
        self.answer = result['output'] 
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=450)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=450, styles={'background-color': '#F6F6F6'}))
        ])
        return pn.WidgetBox(*self.panels, scroll=True)


    def clr_history(self,count=0):
        self.chat_history = []
        return 

In [44]:
cb = cbfs(tools)

inp = pn.widgets.TextInput( placeholder='Enter text here…')

conversation = pn.bind(cb.convchain, inp) 

tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=400),
    pn.layout.Divider(),
)

dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# QnA_Bot')),
    pn.Tabs(('Conversation', tab1))
)
dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'4026cc88-1331-4ec9-b1b6-0305b53117f8': {'version…

C:\Users\Michela\AppData\Local\Temp\ipykernel_5296\2904686110.py:32: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc_time = datetime.datetime.utcnow()
C:\Users\Michela\AppData\Local\Temp\ipykernel_5296\2904686110.py:32: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc_time = datetime.datetime.utcnow()


## Conclusion
This notebook demonstrates `the creation and use of a conversational agent that can perform various tasks using LangChain, OpenAI, and different tools`. By following the examples provided, you can extend the agent's capabilities to suit your specific needs.